### Testing RAG Applications 📑

In [2]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.document import Document
from langchain_ollama import ChatOllama

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
llm = ChatOllama(
    base_url="http://localhost:11434",
    model = "qwen2.5:latest",
    temperature=0.5,
    max_tokens = 250
)

In [4]:
# Load data from Web
loader = WebBaseLoader("https://www.descope.com/learn/post/mcp")
data = loader.load()

# Split text into documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)

# Add text to vector db
embedding = OllamaEmbeddings(model="llama3.2:latest")
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

# Create a retriever
retriever = vectordb.as_retriever()

def format_docs(docs: List[Document]) -> str:
    return "\n\n".join([d.page_content for d in docs])


template = """Answer the question based only on the following context:

    {context}
    
    Give a summary not the full detail

    Question: {question}
    """
prompt = ChatPromptTemplate.from_template(template)


def retrieve_and_format(question):
    docs = retriever.get_relevant_documents(question)
    return format_docs(docs)

chain = {"context": retrieve_and_format, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()


In [5]:
response = chain.invoke("What is MCP")

print(response)

/var/folders/jc/c7p4f2sd36xbqwkp2djn8flc0000gn/T/ipykernel_42468/1491807655.py:32: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(question)


MCP, or Model Context Protocol, is a protocol designed to enable AI assistants to interact with various external APIs and platforms. It supports actions like retrieving channel history from messaging apps, creating forks or branches on GitHub, and more. MCP servers demonstrate how AI assistants can access diverse information sources, and the ecosystem includes reference, official integrations, and community servers, showcasing different implementations of the protocol.


In [6]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [7]:
def query_with_context(question):
    retrieved_document = retrieve_and_format(question)
    response = qa_chain.run(question)
    return response, retrieved_document

In [8]:
actual, context = query_with_context("What is MCP")

actual, context

/var/folders/jc/c7p4f2sd36xbqwkp2djn8flc0000gn/T/ipykernel_42468/1524904417.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(question)


('MCP stands for Model Context Protocol. It is a protocol designed to facilitate interactions between AI assistants and various external services or platforms, including popular messaging apps and GitHub. This protocol supports a wide range of actions such as retrieving channel history from messaging apps and performing operations on GitHub like creating forks or branches and making pull requests. The MCP ecosystem includes reference servers created by the protocol maintainers, official integrations maintained by companies for their platforms, and community-developed servers.',
 "reactions, retrieve channel history, and more. While straightforward, this underscores the potential for MCP to retrieve information from a wide variety of sources, including popular messaging apps.GitHub: Provides support for a wide variety of actions, including creating forks or branches, listing issues, making pull requests, and even searching for code across GItHub repositories. The GitHub MCP server serve

### Testing RAG Application with RAGAs


In [9]:
test_data = [
    {
        "input": "What is MCP",
        "reference": "The Model Context Protocol (MCP) addresses this challenge by providing a standardized way for LLMs to connect with external data sources and tools—essentially a “universal remote” for AI apps. Released by Anthropic as an open-source protocol, MCP builds on existing function calling by eliminating the need for custom integration between LLMs and other apps."
    },
    {
        "input": "What is Relationship between function calling & Model Context Protocol",
        "reference": "The Model Context Protocol (MCP) builds on top of function calling, a well-established feature that allows large language models (LLMs) to invoke predetermined functions based on user requests. MCP simplifies and standardizes the development process by connecting AI applications to context while leveraging function calling to make API interactions more consistent across different applications and model vendors."
    },
    {
        "input": "What are the core components of MCP, just give the heading",
        "reference":""" 
                    - MCP Client
                    - MCP Servers
                    - Protocol Handshake
                    - Capability Discovery
                """
    }
]

In [10]:
dataset = []

for question in test_data:
    actual, context = query_with_context(question['input'])
    
    dataset.append({
        "user_input": question['input'],
        "retrieved_contexts": [context],
        "response": actual,
        "reference": question['reference']
    })

dataset


[{'user_input': 'What is MCP',
  'retrieved_contexts': ["reactions, retrieve channel history, and more. While straightforward, this underscores the potential for MCP to retrieve information from a wide variety of sources, including popular messaging apps.GitHub: Provides support for a wide variety of actions, including creating forks or branches, listing issues, making pull requests, and even searching for code across GItHub repositories. The GitHub MCP server serves as a benchmark for how AI assistants can interact with external APIs.Official MCP\n\nLangChain adapters, and platforms like Superinterface, which helps developers build in-app AI assistants with MCP functionality.Examples of MCP serversThe MCP ecosystem comprises a diverse range of servers including reference servers (created by the protocol maintainers as implementation examples), official integrations (maintained by companies for their platforms), and community servers (developed by independent contributors).Reference se

In [ ]:
from ragas.metrics import LLMContextRecall, NoiseSensitivity, Faithfulness, FactualCorrectness, AnswerRelevancy
from ragas.llms import LangchainLLMWrapper
from ragas import (EvaluationDataset, evaluate)


evaluator_llm = LangchainLLMWrapper(llm)

evaluation_dataset = EvaluationDataset.from_list(dataset)

result = evaluate(dataset=evaluation_dataset, 
                  metrics=[LLMContextRecall(),
                           Faithfulness(),
                           AnswerRelevancy(),
                           FactualCorrectness()],
                  llm = evaluator_llm)

Evaluating: 100%|██████████| 12/12 [02:37<00:00, 13.16s/it]


In [18]:
result.to_pandas()

,user_input,retrieved_contexts,response,reference,context_recall,faithfulness,answer_relevancy,factual_correctness(mode=f1)
0,What is MCP,"[reactions, retrieve channel history, and more...","MCP, or Model Context Protocol, is a protocol ...",The Model Context Protocol (MCP) addresses thi...,0.0,1.0,0.849991,0.17
1,What is Relationship between function calling ...,[then make the API call with it. While this al...,"Function calling, which allows Large Language ...",The Model Context Protocol (MCP) builds on top...,1.0,1.0,0.832278,1.00
2,"What are the core components of MCP, just give...",[understand that MCP doesn’t solve the NxM pro...,MCP Client & Server Ecosystem,\n - MCP Client\n ...,1.0,1.0,0.833868,0.50
